# Part 2

In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt 
import scipy
import scipy.io as spio
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import math
import urllib
from urllib.request import urlopen
%matplotlib inline
import urllib.request
import datetime as dt
from datetime import datetime
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import pandas as pd

In [204]:
def generate_data(m,n):
    data = np.random.uniform(low=-1, high=1, size=(m,(n+1)))
    data = np.sign(data)
    return data

def generate_data_winnow(m,n):
    data = np.random.randint(2, size=(m,(n+1)))
    data = np.sign(data)
    return data

def add_bias(x):
    x_with_bias = np.ones((x.shape[0],x.shape[1]+1))
    x_with_bias[:,:-1] = x
    return x_with_bias

In [8]:
df = generate_data(4,3)

In [9]:
df

array([[ 1., -1., -1.,  1.],
       [ 1.,  1.,  1., -1.],
       [ 1.,  1., -1.,  1.],
       [-1.,  1.,  1., -1.]])

## Binary Perceptron

In [268]:
# Binary perceptron algorithm 
# Learning rate? 
def perceptron_train(x,y,max_epoch=20, tol=0.01, learning_rate=1,random_select=False):
    m = x.shape[0] #number of examples
    n = x.shape[1] #number of features
    #x = add_bias(x)
    error_per_epoch = np.zeros(max_epoch)
    errors = np.zeros(m)
    W = np.zeros(n+1)
    
    num_errors = 0 
    
    if random_select:
        max_epoch =1
        m = np.random.randint(m) #select random number of
    
    for epoch in range(max_epoch):
        #print('epoch=',epoch)
        errors = np.zeros(m)
        num_errors = 0 #This should be bounded..? Maybe calculate the bound in the explanation
    
        #iterate through training set
        for t in range(m):
            x_t = x[t,:]
            y_t = y[t]
            pred_t = np.sign(W@x_t) #0 to begin with 
    
            if pred_t*y_t<=0:
                #print('t=',t)
                #print('error! y_lab=', y_t)
                num_errors +=1
                W = W + learning_rate*x_t.T*y_t
                errors[t] = num_errors 
                #print('W=',W)
                
        error_per_epoch[epoch] = num_errors
        #print(epoch, 'error=',errors[-1])
        
        #print('current_error_rate', error_per_epoch)
        
        if epoch>1:
            #print('old_error=',error_per_epoch[epoch-1])
            #print('new_error=',error_per_epoch[epoch])
            diff_rates = (error_per_epoch[epoch-1] - error_per_epoch[epoch])/m
 
            #Stop if the error rate has increased, 
            #or the difference in error rate between the previous one and the current one < tolerance. 
            if error_per_epoch[epoch]/m < 0.05:
                print('error rate< 10%')
                if diff_rates<tol:
                    print('converging')
                elif diff_rates<0:
                    print('error increased')
                elif error_per_epoch[epoch]==0:
                    print('no error')
                    break
        
    return W, error_per_epoch[:epoch+1]


def perceptron_test(x_test,y_test,W):
    x_test = add_bias(x_test)
    m_test = x_test.shape[0]
    pred = np.sign(x_test@W) #(n,1) x (m_test,n)
    diff = pred - y_test
    mistakes = len(diff[diff!=0])   
    return mistakes,pred

In [265]:
df = generate_data(40,30)
x = df[:,:]
y = df[:,0]
W, error_per_epoch = perceptron_train(x,y,max_epoch=30, tol=0.01, learning_rate=1)

error rate< 10%
error rate< 10%
no error


In [132]:
mistakes,pred = perceptron_test(x[:10],y[:10],W)

In [126]:
error_per_epoch
### Does not seem to converge for some combination of x and y. 

array([28., 19., 15., 14., 11.,  8., 11.,  8., 14.,  7.,  7.,  8.,  7.,
        5.,  8.,  6.,  3.,  7.,  0.])

## Winnow

In [248]:
def winnow_train(x,y,max_epoch=20,tol=0.01, learning_rate=1,gamma_bound=0.5):
    m = x.shape[0] #number of examples
    n = x.shape[1] #number of features
    error_per_epoch = np.zeros(max_epoch)
    errors = np.zeros(m)
    W = np.ones(n)
    mistake_bound = (n/gamma_bound)**2
    gamma_bound = n/2
    num_errors = 0 
    
    #for epoch in range(max_epoch):
    epoch = 0
    while epoch < max_epoch:   
        errors = np.zeros(m)
        num_errors = 0 #This should be bounded..? Maybe calculate the bound in the explanation
        print('epoch=',epoch)
        #iterate through training set
        for t in range(m):
            
            x_t = x[t,:]
            y_t = y[t]
            gamma = W@x_t
            
            if gamma>= gamma_bound:
                pred_t = 1
            else:
                pred_t = 0
            
            if pred_t != y_t:
#                 print('error!')
                #print('pred=',pred_t)
#                 print('y_t=',y_t)
                num_errors+=1
                print('x_t=',x_t)
                print('y_t=',y_t)
                power = (y_t - pred_t)*x_t/1.0 #1 when error made on positive class, -1 when error made on negative class
                print('ratio of update of W',power)
                W = W*(2**power) #element wise multiplication, and element wise power
                print('W=',W)
                #Once weight>n, it should no longer change. 
                W[W>n] = n
#                 print(W)
                errors[t] = num_errors
#                 print('current error',num_errors)
                #print('W=',W)
                
        error_per_epoch[epoch] = num_errors
        
        if epoch>1:
            diff_rates = (error_per_epoch[epoch-1] - error_per_epoch[epoch])/m
            print("Error rate is now ", error_per_epoch[epoch]/m )
            #Stop if the error rate has increased, 
            #or the difference in error rate between the previous one and the current one < tolerance. 
            if error_per_epoch[epoch]/m < 0.05:
                print('error rate< 5%')
                if diff_rates<tol:
                    print('converging')
                elif diff_rates<0:
                    print('error increased')
                elif error_per_epoch[epoch]==0:
                    print('no error')
                    
                break
        epoch += 1
    return W, error_per_epoch[:epoch+1]


def winnow_test(x_test,y_test,W,gamma=0):
    m_test = x_test.shape[0]
    gamma = W@x_t
    if gamma>=n:
        pred_t = 1
    else:
        pred_t = -1
    diff = pred - y_test
    mistakes = len(diff[diff!=0])  
    return mistakes,pred

In [266]:
df = generate_data_winnow(40,2)
x = df
y = df[:,0]
W, error_per_epoch = winnow_train(x,y,max_epoch=15,tol=0.01, learning_rate=1,gamma_bound=0.5)

epoch= 0
x_t= [0 1 1]
y_t= 0
ratio of update of W [ 0. -1. -1.]
W= [1.  0.5 0.5]
x_t= [1 0 0]
y_t= 1
ratio of update of W [1. 0. 0.]
W= [2.  0.5 0.5]
epoch= 1
epoch= 2
Error rate is now  0.0
error rate< 5%
converging


In [267]:
error_per_epoch

array([2., 0., 0.])

In [252]:
W

array([1.95312500e-03, 1.00000000e+00, 5.00000000e-01, 1.00000000e+00,
       9.76562500e-04, 2.00000000e+00, 4.88281250e-04, 4.88281250e-04,
       1.56250000e-02, 1.00000000e+00, 2.50000000e-01, 4.76837158e-07,
       7.81250000e-03, 3.12500000e-02, 1.56250000e-02, 1.52587891e-05,
       2.50000000e-01, 1.25000000e-01, 4.00000000e+00, 2.50000000e-01,
       6.10351562e-05, 2.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 6.25000000e-02, 3.12500000e-02, 1.25000000e-01,
       9.76562500e-04, 1.56250000e-02])

In [ ]:
# Least Squares Regression

In [ ]:
# 1 Nearest Neighbors